## Business Understanding

### Project Goal
The goal of this project is to analyze aviation accident data to identify low-risk aircraft options for a company seeking to expand into the aviation industry. By understanding trends and factors related to aviation accidents, we aim to make data-driven recommendations that will help minimize risk in future aircraft acquisitions.

### Stakeholder
The primary stakeholder for this project is the head of the new aviation division. This individual is responsible for making strategic decisions about which aircraft to purchase for commercial and private use. The stakeholder does not have a technical background, so clear, actionable insights will be crucial.

### Business Problem
The company is venturing into a new line of business—owning and operating aircraft. However, they currently lack internal knowledge of aviation risks. They are looking to determine which aircraft types, manufacturers, or other characteristics are associated with the lowest accident rates. 

### Key Business Questions
- Which aircraft types and manufacturers have the lowest incident and accident rates?
- Are there any common patterns or risk factors among higher-risk aircraft?
- How do accident trends vary over time or by location?
- What recommendations can we make to guide aircraft acquisition based on risk?

Our analysis will lead to three specific business recommendations based on these questions, helping the aviation division make informed and safe purchasing decisions.
